In [ ]:
import torch
from tqdm import tqdm

In [ ]:
def save_checkpoint(state, checkpoint_filename, checkpoint_index=None):
  torch.save(state, checkpoint_filename)
  if checkpoint_index is not None:
    torch.save(state, checkpoint_filename.replace(".pth.tar", str(checkpoint_index) + ".pth.tar"))
  print("[INFO] Checkpoint saved")

In [ ]:
def load_checkpoint(checkpoint, model, optimizer_class, optimizer_mask):
  model.load_state_dict(checkpoint["state_dict"])
  optimizer_class.load_state_dict(checkpoint["optimizer_class"])
  optimizer_mask.load_state_dict(checkpoint["optimizer_mask"])
  print("[INFO] Checkpoint loaded")

In [ ]:
def log_epoch(title, epoch, classification_acc, f1_score, classification_dice, segmentation_acc, segmentation_dice, train_log_filename): 
  f = open(train_log_filename, 'a+') 
  f.write(f"\n[{title}] Epoch {epoch}:\
  \n[Classification] Accuracy: {classification_acc:.2f}, Dice Score: {classification_dice:.2f}, F1 Score: {f1_score:.2f}\
  \n[Segmentation]   Accuracy: {segmentation_acc:.2f}, Dice Score {segmentation_dice:.2f}") 
  f.close()

In [ ]:
def log_train(epoch, classification_loss, segmentation_loss, classification_acc, segmentation_acc, classification_dice, segmentation_dice, train_log_filename):
  f = open(train_log_filename, 'a+') 
  f.write(f"\n[TRAIN] Epoch {epoch}:\
  \n[Classification] Loss: {classification_loss:.2f}, Accuracy: {classification_acc:.2f}, Dice Score: {classification_dice:.2f}\
  \n[Segmentation]   Loss: {segmentation_loss:.2f}, Accuracy: {segmentation_acc:.2f}, Dice Score: {segmentation_dice:.2f}")
  f.close()

In [ ]:
def evaluate_model_incorporate (loader, model, device):
  tp = tn = fp = fn = 0
  num_correct = num_pixels = 0
  dice_score = count_dice = 0
  model.eval()

  with torch.no_grad():
    for x, y1, y2 in tqdm(loader):
      x = x.to(device)
      y1 = y1.long().to(device)
      y2 = y2.float().unsqueeze(1).to(device)
      
      out1, out2 = model(x)

      pred1 = out1.argmax(dim=1, keepdim=True)
      truth1 = y1.view_as(pred1)
      tp += (pred1 * truth1).sum()
      tn += ((1 - pred1).abs() * (1 - truth1).abs()).sum()
      fp += (pred1 * (1 - truth1).abs()).sum()
      fn += ((1 - pred1).abs() * truth1).sum()

      pred2 = torch.sigmoid(out2)
      pred2 = (pred2 > 0.5).float()
      num_correct += (pred2[truth1==1] == y2[truth1==1]).sum()
      num_pixels += torch.numel(pred2[truth1==1])

      if torch.sum(truth1==1) > 0:
        dice_score += (2 * (pred2[truth1==1] * y2[truth1==1]).sum()) / ((pred2[truth1==1] + y2[truth1==1]).sum())
        count_dice += 1

  percision = tp / (tp + fp)
  recall = tp / (tp + fn)

  f1 = 2 * percision * recall / (percision + recall) 
  accuracy_class = (tp + tn) / (tp + tn + fp + fn) * 100
  dice_class = tp / (fp + fn + tp) * 100

  if count_dice > 0:
    accuracy_mask = num_correct/num_pixels*100
    dice_mask = dice_score/count_dice*100
  else:
    accuracy_mask = 0
    dice_mask = 0

  print("\n[EVAL]")
  print(f"[Classification]: Dice: {dice_class:2f}, Acc: {accuracy_class:2f}, F1: {f1:2f},")
  print(f"[Segmentation]:   Dice: {dice_mask:2f},  Acc: {accuracy_mask:2f}")
  print(f"[Confusion]:      TP: {tp.item()}, TN: {tn.item()}, FP: {fp.item()}, FN: {fn.item()}")

  return f1, dice_class, accuracy_class, dice_mask, accuracy_mask

In [ ]:
def evaluate_model_single (loader, model, device):
  tp = tn = fp = fn = 0
  model.eval()

  with torch.no_grad():
    for x, y1, y2 in tqdm(loader):
      x = x.to(device)
      y1 = y1.long().to(device)
      
      out1 = model(x)

      pred1 = out1.argmax(dim=1, keepdim=True)
      truth1 = y1.view_as(pred1)
      tp += (pred1 * truth1).sum()
      tn += ((1 - pred1).abs() * (1 - truth1).abs()).sum()
      fp += (pred1 * (1 - truth1).abs()).sum()
      fn += ((1 - pred1).abs() * truth1).sum()

  percision = tp / (tp + fp)
  recall = tp / (tp + fn)

  f1 = 2 * percision * recall / (percision + recall) 
  accuracy_class = (tp + tn) / (tp + tn + fp + fn) * 100
  dice_class = tp / (fp + fn + tp) * 100

  print("\n[EVAL]")
  print(f"[Classification]: Dice: {dice_class:2f}, Acc: {accuracy_class:2f}, F1: {f1:2f},")
  print(f"[Confusion]:      TP: {tp.item()}, TN: {tn.item()}, FP: {fp.item()}, FN: {fn.item()}")

  dice_mask = accuracy_mask = 0.0

  return f1, dice_class, accuracy_class, dice_mask, accuracy_mask